In [1]:
import os
train_path = '../../raw_train_artifact'
test_path = '../../raw_test_artifact'
embedding_path = '../../embedding_artifact'
input_path = '../../input_artifact'
input_split_path = '../../input_artifact/input_split'
model_path = '../../model_artifact'
output_path = '../../output_artifact'

In [1]:
import sys
import gc
gc.enable()
import time
import re

import numpy as np
import pandas as pd
pd.set_option('display.max_columns',120)
pd.set_option('display.max_rows',2000)
pd.set_option('precision',5)
pd.set_option('float_format', '{:.5f}'.format)

import tqdm
import joblib
import json

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, roc_auc_score
from gensim.models import Word2Vec
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
import logging

log_path = '[1.1]Hugging Face Transformer Experiment.log'
    
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s %(levelname)-s: %(message)s', datefmt='%H:%M:%S')

fh = logging.FileHandler(log_path)
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
logger.addHandler(fh)

sh = logging.StreamHandler(sys.stdout)
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
logger.addHandler(sh)

logger.info(f'Restart notebook\n==========================\n{time.ctime()}\n==========================')

11:43:46 INFO: Restart notebook
Tue Jun  2 11:43:46 2020


In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info('Device in Use: {}'.format(DEVICE))
torch.cuda.empty_cache()
t = torch.cuda.get_device_properties(DEVICE).total_memory/1024**3
c = torch.cuda.memory_cached(DEVICE)/1024**3
a = torch.cuda.memory_allocated(DEVICE)/1024**3
logger.info('CUDA Memory: Total {:.2f} GB, Cached {:.2f} GB, Allocated {:.2f} GB'.format(t,c,a))

11:43:46 INFO: Device in Use: cuda
11:43:46 INFO: CUDA Memory: Total 8.00 GB, Cached 0.00 GB, Allocated 0.00 GB


## Experiment with torch padding

In [62]:
inp_1 = torch.from_numpy(np.ones((3, 120))).float()
inp_2 = torch.from_numpy(np.ones((1, 120))).float()
inp = [inp_1, inp_2]
inp_len = [t.shape[0] for t in inp]
pad_inp = torch.nn.utils.rnn.pad_sequence(inp, batch_first=True, padding_value=0)
print(inp_len, pad_inp.shape)

[3, 1] torch.Size([2, 3, 120])


In [71]:
pad_inp.permute(1,0,2)

torch.Size([3, 2, 120])

In [73]:
lstm = torch.nn.LSTM(input_size=120, hidden_size=128, num_layers=2, bias=True, batch_first=True, bidirectional=True)
out = lstm(pad_inp)
print(out[0].shape)

torch.Size([2, 3, 256])


In [74]:
lstm = torch.nn.LSTM(input_size=120, hidden_size=128, num_layers=2, bias=True, bidirectional=True)
out = lstm(pad_inp.permute(1,0,2))
print(out[0].shape)

torch.Size([3, 2, 256])


## Experiment

In [38]:
del configuration, model
gc.collect()

3860

In [37]:
from transformers import AlbertConfig, AlbertModel

In [40]:
configuration = AlbertConfig()
model = AlbertModel(configuration)

In [41]:
inp = torch.from_numpy(np.concatenate([np.ones((10, 5, 128)), np.zeros((10, 5, 128))], axis=1)).float()

In [42]:
inp.shape

torch.Size([10, 10, 128])

In [43]:
inp[0]

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [44]:
out= model(inputs_embeds=inp)

In [45]:
out[0].shape

torch.Size([10, 10, 4096])

In [48]:
out[0][1].detach().numpy()

array([[-1.0415586 , -0.700448  , -0.23963146, ..., -0.34111258,
        -0.88970816,  2.0453863 ],
       [-1.0419453 , -0.7001726 , -0.24012905, ..., -0.34073353,
        -0.8900057 ,  2.0456576 ],
       [-1.0417546 , -0.7000878 , -0.24050954, ..., -0.34136513,
        -0.8902673 ,  2.0448096 ],
       ...,
       [-1.0415502 , -0.70019853, -0.23981182, ..., -0.34107703,
        -0.8901159 ,  2.0457187 ],
       [-1.0413502 , -0.6995464 , -0.240383  , ..., -0.34042382,
        -0.8905616 ,  2.0456984 ],
       [-1.0413843 , -0.70032   , -0.23944165, ..., -0.34181798,
        -0.89004594,  2.0447192 ]], dtype=float32)